In [7]:
import pyodbc
import pandas as pd

# Figure out which sprocs are missing

## SQL Client

In [18]:
class Client(object):

    DRIVER = '{ODBC Driver 18 for SQL Server}'

    def __init__(self, server, database, username, password):
        self.conn = Client._connect(server, database, username, password)

    @staticmethod
    def _connect(server, database, username, password):
        connection_string = Client._get_connection_string(server, database, username, password)
        return pyodbc.connect(connection_string)

    @staticmethod
    def _get_connection_string(server, database, username, password):
        return f'DRIVER={Client.DRIVER};SERVER={server};DATABASE={database};UID={username};PWD={password}'

    def _execute(self, query):
        cursor = self.conn.cursor()
        cursor.execute(query)
        return cursor
        
    def get_all_query_results(self, query):
        cursor = self._execute(query)
        return cursor.fetchall()


In [19]:
TARGET_SERVER = 'sql-sqldbpoc-dev-centralus-lacewing.database.windows.net'
TARGET_DATABASE = 'IPEP_C3LX_Training'
TARGET_USERNAME = 'sql-sqldbpoc-dev-centralus-lacewing-admin'
TARGET_PASSWORD = ')D&{F3T*(51_@8*4-mOp'

In [20]:
target_client = Client(TARGET_SERVER, TARGET_DATABASE, TARGET_USERNAME, TARGET_PASSWORD)

## List sprocs in target database

In [21]:
LIST_SPROCS = """
SELECT routines.SPECIFIC_SCHEMA, routines.SPECIFIC_NAME, modules.definition
FROM sys.sql_modules modules
join INFORMATION_SCHEMA.ROUTINES routines
on modules.object_id = object_id(routines.SPECIFIC_SCHEMA + '.' + routines.SPECIFIC_NAME)
"""

In [23]:
results = target_client.get_all_query_results(LIST_SPROCS)

In [24]:
target_df = pd.DataFrame.from_records(results)

In [25]:
target_df.columns = ['schema', 'sproc', 'ddl']

In [26]:
target_df

,schema,sproc,ddl
0,dbo,SP_ReportEnrollmentDetails,\nCREATE procedure [dbo].[SP_ReportEnrollmentD...
1,dbo,SP_ReportEnrollmentStatusWiseCount,\nCREATE PROCEDURE SP_ReportEnrollmentStatusWi...
2,dbo,SP_ReportInterventionDetails,\nCREATE procedure [dbo].[SP_ReportInterventio...
3,dbo,SP_ReportInterventionStatusCount,\nCREATE PROCEDURE SP_ReportInterventionStatus...
4,dbo,SP_Trn_ActivityAssessmentResponse,\nCreate procedure SP_Trn_ActivityAssessmentRe...
...,...,...,...
81,dbo,SP_PatientAppointments,\nCreate procedure SP_PatientAppointments (\r\...
82,dbo,SP_ReportCallsDetails,\nCREATE procedure [dbo].[SP_ReportCallsDetail...
83,dbo,SP_ReportCallStatusCount,\nCREATE PROCEDURE SP_ReportCallStatusCount\r\...
84,dbo,SP_ReportConsultDetails,\nCREATE procedure SP_ReportConsultDetails (\r...


## Read exported data from source database (database too old to be queried using odbc)

In [27]:
source_df = pd.read_csv('./ipep-source-db-sprocs.csv', header=None)
source_df.columns = ['schema', 'sproc', 'ddl']

In [28]:
source_df

,schema,sproc,ddl
0,dbo,USP_Get_UHCNDC_MedList,Create procedure USP_Get_UHCNDC_MedList AS B...
1,dbo,USP_Trn_Patient_Pharmcy,Create procedure USP_Trn_Patient_Pharmcy (@Pa...
2,dbo,USP_GetAccountWiseActivityList,Create procedure [dbo].[USP_GetAccountWiseAct...
3,dbo,USP_Trn_Patient_Prescriber,Create procedure USP_Trn_Patient_Prescriber (...
4,dbo,USP_GetAllBillingCodeFees,Create procedure USP_GetAllBillingCodeFees A...
...,...,...,...
240,dbo,USP_ServiceBillingSummaryReport,CREATE procedure USP_ServiceBillingSummaryRep...
241,dbo,USP_Get_All_Trn_BookAppointment,create procedure USP_Get_All_Trn_BookAppointm...
242,dbo,USP_Trn_Patient_Contact,Create procedure USP_Trn_Patient_Contact (@Pa...
243,dbo,USP_Get_Trn_BookAppointment,create procedure USP_Get_Trn_BookAppointment ...


In [30]:
missing_sprocs = set(source_df.sproc).difference(set(target_df.sproc))

In [32]:
missing_df = source_df[source_df.sproc.isin(missing_sprocs)]    

## We are missing 159 sprocs

In [45]:
missing_df.shape

(159, 3)

# Try to migrate by executing ddl

## Write ddl to file so we can sqlfluff it

In [39]:
for index, row in missing_df.iterrows():
    with open(f"./ddl/{row['sproc']}.sql", 'w') as file:
        file.write(row['ddl'])

## Try creating as many of the existing sprocs as possible

In [42]:
for index, row in missing_df.iterrows():
    with open(f"./ddl/{row['sproc']}.sql", 'r') as file:
        ddl = file.read()

    try:
        target_client.get_all_query_results(ddl)
    except Exception as e:
        print("*"*100)
        print(e)
        print(row)

****************************************************************************************************
('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near 'BEGIN'. (102) (SQLExecDirectW)")
schema                                                  dbo
sproc                                USP_Get_UHCNDC_MedList
ddl        Create procedure USP_Get_UHCNDC_MedList AS  B...
Name: 0, dtype: object
****************************************************************************************************
No results.  Previous SQL was not a query.
schema                                                  dbo
sproc                               USP_Trn_Patient_Pharmcy
ddl        Create procedure USP_Trn_Patient_Pharmcy (@Pa...
Name: 1, dtype: object
****************************************************************************************************
No results.  Previous SQL was not a query.
schema                                                  dbo
sproc           

## Check to see how many new sprocs we got (not that many)

In [43]:
results2 = target_client.get_all_query_results(LIST_SPROCS)

In [44]:
len(results), len(results2)

(86, 93)